# 2019 Novel Coronavirus (SARS-CoV-2) and COVID-19 Unpivoted Data

The following script takes data from the repository of the 2019 Novel Coronavirus Visual Dashboard operated by Johns Hopkins University's Center for Systems Science and Engineering (JHU CSSE). It will apply necessary cleansing/reformatting to make it use in traditional relational databases and data visualization tools.

In [ ]:
import pandas as pd
import os
from datetime import datetime
import pycountry
from copy import deepcopy

In [ ]:
# papermill parameters
output_folder = "../output/"

Data downloaded directly from Johns Hopkins git repository, located at: https://github.com/CSSEGISandData/COVID-19. Their repository has three different CSV files – one each for `confirmed`, `deaths` and `recovered` data. The data is keyed into an array of `pandas` `DataFrame`s.

In [ ]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv",keep_default_na=False)
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv",keep_default_na=False)
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv",keep_default_na=False)

confirmed["Case_Type"] = "Confirmed"
deaths["Case_Type"] = "Deaths"
recovered["Case_Type"] = "Recovered"

key_columns = ["Country/Region",
               "Province/State",
               "Lat",
               "Long",
               "Case_Type"]

data = [confirmed, deaths, recovered]

The original dataset stores the number of `Cases` for a given day in columns. 
This is not useful for reporting, thus we move these date columns to rows:

In [ ]:
def unpivot(df):
    # unpivot all non-key columns
    melted = df.melt(id_vars=key_columns, var_name="Date", value_name="Cases")
    # change our new Date field to Date type
    melted["Date"]= pd.to_datetime(melted["Date"]) 
    
    return melted

unpivoted_data = list(map(unpivot, data))

## Data Quality

We are recombining the data set first to use county-level data aggregates before 09 March and state-level data thereafter. Because the data for `US-VI` (U.S. Virgin Islands) still contains a comma in data after March 10, we are executing a substitution before filtering.

In [ ]:
def drop_incorrect_county_state_data(df):
    stateBeforeMarch9th = df[ (df["Date"] <= "2020-03-09") & (df["Country/Region"] == "US") & (df["Province/State"].str.contains(",") == False) ].index
    countyAfterMarch10th = df[ (df["Date"] > "2020-03-09") & (df["Country/Region"] == "US") & df["Province/State"].str.contains(",") ].index

    return df.drop(stateBeforeMarch9th).drop(countyAfterMarch10th)

unpivoted_data = [df.replace({"Virgin Islands, U.S.": "Virgin Islands"}) for df in unpivoted_data]
unpivoted_data = [drop_incorrect_county_state_data(df) for df in unpivoted_data]

We normalize data on the Virgin Islands and Washington D.C. in the following step.

In [ ]:
locality_replacements = {"Washington, D.C.": "District of Columbia",
                         "Virgin Islands, U.S.": "Virgin Islands, VI"}

def replace_localities(df):
    return df.replace(locality_replacements)

unpivoted_data = [replace_localities(df) for df in unpivoted_data]

Next, we resolve the geographies of U.S. counties to their respective states (`Province/State`): 

In [ ]:
subdivisions = {i.name: i.code for i in pycountry.subdivisions.get(country_code="US")}
abbreviations = {subdivisions[k]: k for k in subdivisions}

def resolve_US_geography(row):
    county, state = row["Province/State"].split(", ")
    state.replace("D.C.", "DC")
    row["Province/State"] = abbreviations["US-" + state.strip()]
    return row
        
def resolve_geography_df(df):
    return df.apply(lambda row: resolve_US_geography(row) if row["Country/Region"] == "US" and row["Province/State"] not in list(subdivisions.keys()) and ", " in row["Province/State"] else row, axis="columns")

unpivoted_data = [resolve_geography_df(df) for df in unpivoted_data]

A number of states have inconsistent naming or special characters, such as `Taiwan*`. These are normalised through a replacement `dict` with ISO3166-1 compliant names. Data is then aggregated for each division by date and case type.

In [ ]:
changed_names = {
    "Holy See": "Holy See (Vatican City State)",
    "Vatican City": "Holy See (Vatican City State)",
    "Hong Kong SAR": "Hong Kong",
    "Iran (Islamic Republic of)": "Iran, Islamic Republic of",
    "Iran": "Iran, Islamic Republic of",
    "Macao SAR": "Macao",
    "Macau": "Macao",
    "Republic of Korea": "Korea, Republic of",
    "South Korea": "Korea, Republic of",
    "Korea, South": "Korea, Republic of",
    "Republic of Moldova": "Moldova, Republic of",
    "Russia": "Russian Federation",
    "Saint Martin": "Sint Maarten (Dutch part)",
    "St. Martin": "Sint Maarten (Dutch part)",
    "Taipei and environs": "Taiwan, Province of China",
    "Vietnam": "Viet Nam",
    "occupied Palestinian territory": "Palestine, State of",
    "Taiwan*": "Taiwan, Province of China",
    "Congo (Brazzaville)": "Congo",
    "Congo (Kinshasa)": "Congo, The Democratic Republic of the",
    "Gambia, The": "Gambia",
    "Tanzania": "Tanzania, United Republic of",
    "US": "United States",
    "Curacao": "Curaçao",
    "Brunei": "Brunei Darussalam",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Moldova": "Moldova, Republic of",
    "The Bahamas": "Bahamas",
    "Venezuela": "Venezuela, Bolivarian Republic of",
    "Bolivia": "Bolivia, Plurinational State of"
}


for idx, df in enumerate(unpivoted_data):
    df["Country/Region"] = df["Country/Region"].replace(changed_names)
    df["Cases"] = df["Cases"].replace('',0).astype(int)
        
    unpivoted_data[idx] = df.groupby(by=["Country/Region","Province/State","Date","Case_Type"], as_index=False) \
        .agg({"Cases": "sum", "Long": "first", "Lat": "first"})

## Adding ISO3166-1 and ISO3166-2 identifiers

To facilitate easy recognition, ISO3166-1 identifiers are added to all countries and ISO3166-2 identifiers are added where appropriate. This is the case where subregional data exists:

* Australia
* Canada
* France (`France` for metropolitan France, separate regions for DOM/TOMs
* PRC
* US
* UK (the `UK` province identifier encompasses only Great Britain and Northern Ireland, other dependencies reporting to the UK authorities are separate subdivisions)
* The Kingdom of the Netherlands (`Netherlands` encompasses the constituent country of the Netherlands, and the other constituent countries register cases as separate provinces of the Kingdom of the Netherlands)

In [ ]:
def resolve_iso3166_1_row(row):
    if row["Country/Region"] is not "Cruise Ship":
        if pycountry.countries.get(name=row["Country/Region"]):
            row["ISO3166-1"] = pycountry.countries.get(name=row["Country/Region"]).alpha_2
        else:
            row["ISO3166-1"] = ""
    return row

def resolve_iso3166_1_df(df):
    return df.apply(resolve_iso3166_1_row, axis="columns")

In [ ]:
unpivoted_data = [resolve_iso3166_1_df(df) for df in unpivoted_data]

In [ ]:
fr_subdivisions = {"France": "FR",
                       "French Guiana": "GF",
                       "French Polynesia": "PF",
                       "Guadeloupe": "GUA",
                       "Mayotte": "YT",
                       "Reunion": "RE",
                       "Saint Barthelemy": "BL",
                       "St Martin": "MF"}

nl_subdivisions = {"Netherlands": "NL",
                   "Aruba": "AW",
                   "Curacao": "CW"}

cn_subdivisions = {'Jilin': 'CN-JL',
 'Xizang': 'CN-XZ',
 'Anhui': 'CN-AH',
 'Jiangsu': 'CN-JS',
 'Yunnan': 'CN-YN',
 'Beijing': 'CN-BJ',
 'Jiangxi': 'CN-JX',
 'Zhejiang': 'CN-ZJ',
 'Chongqing': 'CN-CQ',
 'Liaoning': 'CN-LN',
 'Fujian': 'CN-FJ',
 'Guangdong': 'CN-GD',
 'Inner Mongolia': 'CN-NM',
 'Gansu': 'CN-GS',
 'Ningxia': 'CN-NX',
 'Guangxi': 'CN-GX',
 'Qinghai': 'CN-QH',
 'Guizhou': 'CN-GZ',
 'Sichuan': 'CN-SC',
 'Henan': 'CN-HA',
 'Shandong': 'CN-SD',
 'Hubei': 'CN-HB',
 'Shanghai': 'CN-SH',
 'Hebei': 'CN-HE',
 'Shaanxi': 'CN-SN',
 'Hainan': 'CN-HI',
 'Shanxi': 'CN-SX',
 'Tianjin': 'CN-TJ',
 'Heilongjiang': 'CN-HL',
 'Hunan': 'CN-HN',
 'Xinjiang': 'CN-XJ',
 'Tibet': "CN-XZ"}

uk_subdivisions = {"United Kingdom": "UK",
                   "Cayman Islands": "KY",
                   "Channel Islands": "CHA",
                   "Gibraltar": "GI",
                   "Montserrat": "MS"}

subdivisions = {
    "AU": {subdivision.name: subdivision.code.replace("AU-", "") for subdivision in pycountry.subdivisions.get(country_code="AU")},
    "CA": {subdivision.name: subdivision.code.replace("CA-", "") for subdivision in pycountry.subdivisions.get(country_code="CA")},
    "US": {subdivision.name: subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions.get(country_code="US")},
    "GB": uk_subdivisions,
    "CN": cn_subdivisions,
    "NL": nl_subdivisions,
    "FR": fr_subdivisions
}

In [ ]:
countries_with_subdivisions = list(subdivisions.keys())

def resolve_iso3166_2_row(row):
    if row["ISO3166-1"] in countries_with_subdivisions:
        row["ISO3166-2"] = subdivisions[row["ISO3166-1"]].get(row["Province/State"])
    else:
        row["ISO3166-2"] = ""
    return row

def resolve_iso3166_2_df(df):
    return df.apply(resolve_iso3166_2_row, axis="columns")

In [ ]:
unpivoted_data = [resolve_iso3166_2_df(df) for df in unpivoted_data]

## Calculating case changes

Next, we sort the data by primary keys and `Date` to ensure we can add a `Differences` column as a window function.

In [ ]:
sorted_data = list(map(lambda df: df.sort_values(by=key_columns + ["Date"], ascending=True), unpivoted_data))

As `Cases` are actual snapshots (running numbers), we define changes as the difference to the previous day's value. In other words, `Difference` equals today's `Cases` minus yesterday's `Cases` for each region/state and each case category.

In [ ]:
for df in sorted_data:
    df["Difference"] = df["Cases"] - df.groupby( key_columns )["Cases"].shift(1, fill_value = 0) 

concated = pd.concat(sorted_data)

## Calculating active cases

Acive cases are confirmed cases that are not deceased or registered as recovered. This is relevant as active cases determine the demands on the healthcare system. We calculate `Active` case types as:

```
Active = Confirmed - Deaths - Recovered
```

As a first step, we merge the different type of cases into a single row for each `Country/Province/Date` keys:

In [ ]:
confirmed = concated[concated["Case_Type"].eq("Confirmed")]
deaths = concated[concated["Case_Type"].eq("Deaths")]
recovered = concated[concated["Case_Type"].eq("Recovered")]

active = confirmed  \
        .merge(deaths, validate= "one_to_one", suffixes =["","_d"], on=["Country/Region","Province/State","Date"]) \
        .merge(recovered, validate= "one_to_one", suffixes =["","_r"], on= ["Country/Region","Province/State","Date"])

Then, we apply the calculations both for `Cases` and `Difference`:

In [ ]:
active["Case_Type"] = "Active"
active["Cases"] = active["Cases"] - active["Cases_r"] - active["Cases_d"]
active["Difference"] = active["Difference"] - active["Difference_r"] - active["Difference_d"]

Finally, we merge the `Active` segment with the original one. 

In [ ]:
data = pd.concat([concated,active], join="inner")

data["Case_Type"].unique()

## Adding timestamp

Before we save the file locally, we add the `Last_Update_Date` in `UTC` time zone.

In [ ]:
data["Last_Update_Date"] = datetime.utcnow()

## Output

Finally, we store the output in the `output` folder as `JHU_COVID-19.csv` as an unindexed CSV file.

In [ ]:
data.to_csv(output_folder + "JHU_COVID-19.csv", index=False)